In [13]:
import sys
sys.path.append('/home/sharedrive/nafi/traffic/lib/python3.9/site-packages')

import pandas as pd
from chronos import ChronosPipeline
import torch
from table_evaluator import TableEvaluator
import matplotlib.pyplot as plt
from sdv.single_table import GaussianCopulaSynthesizer
from sdv.metadata import Metadata
import numpy as np
from ctgan import CTGAN
from ctgan import load_demo
from sklearn.preprocessing import StandardScaler, LabelEncoder, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, classification_report
import xgboost as xgb
from table_evaluator import TableEvaluator
import pickle
import os

In [14]:
itu_test_df = pd.read_csv("/home/sharedrive/nafi/trafficp3/datasets/feature_selected_itu_test.csv")
itu_test_df.shape

(511540, 25)

In [15]:
itu_test_df.drop(columns=["Init_Win_bytes_forward", "Flow_IAT_Min"], inplace=True)
itu_test_df.shape


(511540, 23)

In [16]:
def load_label_encoder(file_name):
    save_path = "/home/sharedrive/nafi/trafficp3/cicidis"
    with open(f'{save_path}/models/{file_name}_label_encoder.pkl', 'rb') as le_file:
        return pickle.load(le_file)

def load_scaler(file_name):
    save_path = "/home/sharedrive/nafi/trafficp3/cicidis"
    with open(f'{save_path}/models/{file_name}_scaler.pkl', 'rb') as scaler_file:
        return pickle.load(scaler_file)

def load_model(file_name, rounds, num_samples=1000, epochi=1500):
    save_path = "/home/sharedrive/nafi/trafficp3/cicidis"
    with open(f'{save_path}/models/{file_name}sample{num_samples}epoch{epochi}rounds{rounds}.pkl', 'rb') as model:
        return pickle.load(model) 


def inference(test_df, load_file_name, rounds, num_sample=1000):
    x_test = test_df.drop(columns=["Label"])
    y_test = test_df["Label"]

    scaler = load_scaler(load_file_name)
    le = load_label_encoder(load_file_name)
    RF_model = load_model(load_file_name, rounds)
    
    x_test = scaler.transform(x_test)
    y_test = le.transform(y_test)
    print("scaler and label encoder loaded & applied!!")


    pred_rf = RF_model.predict(x_test)

    print(f"Classification Report for Sample Size {num_sample}:")
    print(classification_report(y_test, pred_rf, digits=4, target_names=le.classes_))

In [17]:
inference(itu_test_df, 'synthetic_from_cicids_ctgan', 1)

scaler and label encoder loaded & applied!!


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    3.1s


Classification Report for Sample Size 1000:
                           precision    recall  f1-score   support

                   BENIGN     0.9279    0.8775    0.9020    410865
                      Bot     0.0100    0.9802    0.0198       354
                     DDoS     0.9992    0.8443    0.9152     23160
            DoS_GoldenEye     0.9892    0.9801    0.9846      1861
                 DoS_Hulk     0.8372    0.9096    0.8719     41626
         DoS_Slowhttptest     1.0000    0.4356    0.6069       994
            DoS_slowloris     0.9950    0.9456    0.9697      1048
              FTP-Patator     0.9937    0.9861    0.9899      1436
               Heartbleed     1.0000    1.0000    1.0000         2
             Infiltration     0.0020    0.5000    0.0040         6
                 PortScan     1.0000    0.0032    0.0065     28728
              SSH-Patator     1.0000    0.4911    0.6587      1067
  Web_Attack__Brute_Force     0.0162    0.9816    0.0319       272
Web_Attack__Sql_I

In [18]:
inference(itu_test_df, 'synthetic_from_cicids_ctgan', 2)

scaler and label encoder loaded & applied!!


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    3.1s


Classification Report for Sample Size 1000:
                           precision    recall  f1-score   support

                   BENIGN     0.9158    0.8924    0.9040    410865
                      Bot     0.0148    0.7175    0.0290       354
                     DDoS     0.8783    0.9390    0.9076     23160
            DoS_GoldenEye     0.9838    0.8157    0.8919      1861
                 DoS_Hulk     0.8675    0.8395    0.8533     41626
         DoS_Slowhttptest     1.0000    0.3652    0.5350       994
            DoS_slowloris     0.9950    0.9475    0.9707      1048
              FTP-Patator     0.9930    0.9861    0.9895      1436
               Heartbleed     1.0000    1.0000    1.0000         2
             Infiltration     0.0017    0.6667    0.0035         6
                 PortScan     0.9906    0.0073    0.0146     28728
              SSH-Patator     1.0000    0.4902    0.6579      1067
  Web_Attack__Brute_Force     0.0291    0.4890    0.0549       272
Web_Attack__Sql_I

In [19]:
inference(itu_test_df, 'synthetic_from_cicids_ctgan', 3)

scaler and label encoder loaded & applied!!


[Parallel(n_jobs=1)]: Done  40 tasks      | elapsed:    3.3s


Classification Report for Sample Size 1000:
                           precision    recall  f1-score   support

                   BENIGN     0.9959    0.9906    0.9932    410865
                      Bot     0.9467    0.6017    0.7358       354
                     DDoS     0.9998    0.9994    0.9996     23160
            DoS_GoldenEye     0.9893    0.9909    0.9901      1861
                 DoS_Hulk     0.9183    0.9778    0.9471     41626
         DoS_Slowhttptest     0.9802    0.9960    0.9880       994
            DoS_slowloris     0.9990    1.0000    0.9995      1048
              FTP-Patator     0.9951    0.9868    0.9909      1436
               Heartbleed     1.0000    1.0000    1.0000         2
             Infiltration     0.8333    0.8333    0.8333         6
                 PortScan     0.9940    0.9999    0.9969     28728
              SSH-Patator     0.9981    0.4967    0.6633      1067
  Web_Attack__Brute_Force     0.9203    0.8493    0.8834       272
Web_Attack__Sql_I